In [2]:
!pip install requests beautifulsoup4 --quiet


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import requests, bs4, json, time, pathlib

# --- CONFIGURA TUS CONSTANTES ---
COMP_ID   = "1410"      # id de la competición en fecanbaloncesto.com
PA_LIGA   = 75          # media pts encajados por partido (aprox.)
DELAY     = 0.5         # segundos entre peticiones (cortesía)

BASE  = "https://www.fecanbaloncesto.com"
API   = BASE + "/services/estadisticas_partido.php?idPartido="
COMP  = f"{BASE}/competicion/?id={COMP_ID}"

# --- FUNCIONES DE SCRAPE Y CÁLCULO ---
def partido_ids():
    html = requests.get(COMP, timeout=10).text
    soup = bs4.BeautifulSoup(html, "html.parser")
    return [a["data-idpartido"] for a in soup.select("a.estadisticas")]

def eff(p):
    FGM = p['t2conv'] + p['t3conv']
    FGA = p['t2int'] + p['t3int']
    return (p['puntos']
            - (FGA-FGM)
            - (p['tlint']-p['tlconv'])
            - p['faltas']
            + 0.1 * p['min'])

def procesa_partido(pid):
    data = requests.get(API+pid, timeout=10).json()
    salida = []
    for side in ('local', 'visitante'):
        PA_rival = data['marcadorVisitante'] if side=='local' else data['marcadorLocal']
        FD = PA_LIGA / PA_rival
        for p in data[side]:
            val  = eff(p) * FD
            cred = max(5, min(15, round(0.8*val + 3)))
            salida.append({
                "partido" : pid,
                "equipo"  : side,
                "nombre"  : p['nombre'],
                "min"     : p['min'],
                "pts"     : p['puntos'],
                "eff"     : round(val, 1),
                "cred"    : cred
            })
    return salida

# --- EJECUTA PARA TODA LA JORNADA ---
todos = []
for pid in partido_ids():
    todos.extend(procesa_partido(pid))
    time.sleep(DELAY)

# Guarda en la carpeta de Carnets (visible en Archivos)
ruta = pathlib.Path.home() / "Documents" / "basket_data.json"
with open(ruta, "w", encoding="utf8") as f:
    json.dump(todos, f, ensure_ascii=False, indent=2)

print(f"✅ Generado {ruta} con {len(todos)} registros")


✅ Generado /private/var/mobile/Containers/Data/Application/E489EA38-32B7-4F3B-B70B-04343FABA908/Documents/basket_data.json con 0 registros


In [5]:
import requests, bs4
print("🎉 Todo OK: módulos disponibles")

🎉 Todo OK: módulos disponibles
